In [ ]:
import cv2
import os
from pathlib import Path
import sys
sys.path.append('..')
from augmentations import CenterCrop

In [28]:
CROP = 1024
MIN_SIZE = 512

### FLICKR

In [29]:
flickr_root = Path('../data/external/flickr_images_orig')
flickr_crop_root = Path('../data/external/flickr_images_{}'.format(CROP))

try:
    flickr_crop_root.mkdir()
except FileExistsError:
    print('!!! Will overwrite files, is it OK?')
    
for d in [d for d in flickr_root.iterdir() if d.is_dir()]:
    (flickr_crop_root / d.stem).mkdir(exist_ok=True)

In [34]:
skip_exists = False

with open(str(flickr_root / 'good_jpgs_refined')) as f:
    for s_path in [l.strip() for l in f.readlines()]:
        # TIF does not loose quality
        d_path = flickr_crop_root / s_path
        d_path = str(d_path).replace('jpg', 'tif')
        if skip_exists and Path(d_path).exists():
            continue
        s_path = str(flickr_root / s_path)
        img = cv2.imread(s_path)
        h, w = img.shape[:2]
        if img.ndim != 3:
            print('{} dim {}, skip'.format(s_path, img.ndim))
        continue
        if h < MIN_SIZE or w < MIN_SIZE:
            print('{} size {}, skip'.format(s_path, (w,h)))
            continue
        sz = min(h, CROP), min(w, CROP)
        img = CenterCrop(sz)(img)
        cv2.imwrite(d_path, img)

### ORG